# Error Analysis of Products
This notebook analyzes buy boxes found on product pages. Specifically, how often the default sellers and shippers switch from Amazon to a third-party.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import glob
import math
import json
from collections import Counter
from multiprocessing import Pool

from tqdm import tqdm
from lxml import html
import numpy as np
import pandas as pd

import parsers as P
from utils import value_counts
from parsers import process_search_result, is_sold_by_amazon, is_shipped_by_amazon

In [3]:
# inputs
fn_prod = '../data/output/datasets/products.csv.xz'
fn_amazon = '../data/output/datasets/amazon_private_label.csv.xz'

fn_prod_new = '../data/output/datasets/products-sample-spotcheck.csv.xz'
pattern_spot_check = '../data/input/spotcheck/selenium-products/*/*/*/2021/05/*/webpage_product.html'

In [4]:
df_amazon = pd.read_csv(fn_amazon, compression='xz')
amazon_asin = df_amazon.asin.tolist()

In [5]:
df_prod = pd.read_csv(fn_prod, compression='xz')
len(df_prod)

157405

## Check product pages

In [6]:
value_counts(df_prod, "is_page_gone")

,count,percentage
False,156209,0.992402
True,1196,0.007598


In [7]:
value_counts(df_prod, "no_buybox_winner")

,count,percentage
False,151805,0.964423
True,5600,0.035577


In [8]:
len(df_prod[(df_prod.is_out_of_stock == True) |
            (df_prod.is_page_gone == True)]) / len(df_prod)

0.03895047806613513

In [9]:
df_prod.is_sold_by_amazon.value_counts(normalize=True, dropna=False)

False    0.615038
True     0.282456
True     0.102506
Name: is_sold_by_amazon, dtype: float64

In [10]:
len(df_prod[df_prod.sold_by.isnull()]) / len(df_prod)

0.10250627362536133

## Check re-collected product pages
We pulled a random sample of valid product pages, and re-collected them a few months later.

In [11]:
# parse the re-collected products.
if not os.path.exists(fn_prod_new):
    files_products = glob.glob(pattern_spot_check)
    product_data = []
    with Pool(processes=32) as pool:
        for record in tqdm(pool.imap_unordered(P.parse_product_page, 
                                               files_products), 
                           total=len(files_products)):
            product_data.append(record)

    df_prod_new = pd.DataFrame(product_data)
    
    df_prod_new["asin"] = df_prod_new.fn.apply(lambda x: x.split('/2021')[0].split('/')[-1])
    valid_asins = df_prod[~df_prod.sold_by.isnull()].asin
    df_prod_new = df_prod_new[df_prod_new.asin.isin(valid_asins)]
    df_prod_new = df_prod_new.sample(2500, random_state=303)
    
    df_prod_new['is_sold_by_amazon'] = df_prod_new.apply(is_sold_by_amazon, axis=1)
    df_prod_new['is_shipped_by_amazon'] = df_prod_new.apply(is_shipped_by_amazon, axis=1)
    
    df_prod_new.to_csv(fn_prod_new, index=False, compression='xz')
else:
    df_prod_new = pd.read_csv(fn_prod_new, compression='xz')

In [12]:
value_counts(df_prod_new, "is_out_of_stock")

,count,percentage
False,2347,0.9388
True,153,0.0612


In [13]:
value_counts(df_prod_new, "is_page_gone")

,count,percentage
False,2461,0.9844
True,39,0.0156


In [14]:
value_counts(df_prod_new, "no_buybox_winner")

,count,percentage
False,2416,0.9664
True,84,0.0336


In [15]:
# what percentage of pages are out of stock or gone
len(df_prod_new[(df_prod_new.is_out_of_stock == True) | 
                (df_prod_new.is_page_gone == True)]) / len(df_prod_new)

0.0768

In [16]:
df_prod_new.fillna(value=np.nan, inplace=True)
df_prod.fillna(value=np.nan, inplace=True)

In [17]:
len(df_prod_new)

2500

In [18]:
# make a dataframe of new and old product page info
df = df_prod[df_prod.asin.isin(df_prod_new.asin.unique())].merge(df_prod_new, on='asin', suffixes= ('_og', '_new'))

In [19]:
amazon_sellers = ['zappos', 'whole foods', 'amazon']

def who_switched(row, col1='sold_by_new', col2='sold_by_og'):
    sold_new = False
    sold_old = False
    if any(seller in str(row[col1]).lower() for seller in amazon_sellers):
        sold_new = True
    if any(seller in str(row[col2]).lower() for seller in amazon_sellers):
        sold_old = True
    
    if sold_new == False and sold_old == False:
        return 'third party'
    
    if sold_new == True and sold_old == False:
        return 'to amazon'
    
    if sold_new == False and sold_old == True:
        return 'to third party'
    
    if sold_new == True and sold_old == True:
        return "amazon"
    
def seller_switch(row):
    return who_switched(row, col1='shipped_by_new', col2='shipped_by_og')

In [20]:
df = df[~(df.sold_by_new.isnull() | df.sold_by_og.isnull())]
len(df)

2103

In [21]:
df.is_sold_by_amazon_og.value_counts(normalize=True)

False    0.697575
True     0.302425
Name: is_sold_by_amazon_og, dtype: float64

In [22]:
df.is_shipped_by_amazon_new.value_counts(normalize=True)

True     0.886353
False    0.113647
Name: is_shipped_by_amazon_new, dtype: float64

In [23]:
df['seller_delta'] = df.apply(who_switched, axis=1)
df['shipper_delta'] = df.apply(seller_switch, axis=1)

In [24]:
# how many changed at all?
(df['sold_by_new'] == df['sold_by_og']).value_counts(normalize=True)

True     0.839277
False    0.160723
dtype: float64

In [25]:
value_counts(df[df.sold_by_new != df.sold_by_og], "seller_delta")

,count,percentage
third party,257,0.760355
to amazon,34,0.100592
to third party,32,0.094675
amazon,15,0.044379


In [26]:
value_counts(df, "seller_delta")

,count,percentage
third party,1433,0.681408
amazon,604,0.287209
to amazon,34,0.016167
to third party,32,0.015216


In [27]:
def get_confidence_interval(perc = 1.5,
                            ss = 25,
                            pop = 100,
                            confidence_level = 95):
    """
    Calculates confidence interval given a percentage, sample size, and population size.
    taken from: https://www.surveysystem.com/sscalc.htm
    see: https://opentextbc.ca/introbusinessstatopenstax/chapter/a-confidence-interval-for-a-population-proportion/
    """
    conf2z = {
        95 : 3.8416,
        99 : 6.6564
    }
    
    zValC = conf2z.get(confidence_level)
    if pop == 0:
        pf = 1
    else:
        pf = (pop - ss) / (pop - 1)

    return math.sqrt(zValC * (perc / 100) * (1 - perc / 100) / ss * pf) * 100   

In [28]:
# seller changes to Amazon
perc_seller_to_amazon = df[df.seller_delta.isin(['to amazon', 'to amazon'])].asin.nunique() / df.asin.nunique() * 100
CI = get_confidence_interval(perc_seller_to_amazon, ss=len(df), pop=df_prod.asin.nunique())
perc_seller_to_amazon, CI
# perc_seller_to_amazon - CI, perc_seller_to_amazon + CI

(1.6167379933428434, 0.5354235980761819)

In [29]:
# seller changes to third-party
perc_seller_to_3p = df[df.seller_delta.isin(['to amazon', 'to third party'])].asin.nunique() / df.asin.nunique() * 100
CI = get_confidence_interval(perc_seller_to_3p, ss=len(df), pop=df_prod.asin.nunique())
perc_seller_to_3p, CI
# perc_seller_to_3p - CI, perc_seller_to_3p + CI

(3.138373751783167, 0.7401935334253164)

In [30]:
# shipper changes to Amazon
perc_shipper_to_amazon = df[df.shipper_delta.isin(['to amazon', 'to amazon'])].asin.nunique() / df.asin.nunique() * 100
CI = get_confidence_interval(perc_shipper_to_amazon, ss=len(df),pop=df_prod.asin.nunique())
perc_shipper_to_amazon, CI
# perc_shipper_to_amazon - CI, perc_shipper_to_amazon + CI

(2.8530670470756063, 0.7067854701849021)

In [31]:
# shipper changes to third-party
perc_shipper_to_3p = df[df.shipper_delta.isin(['to amazon', 'to third party'])].asin.nunique() / df.asin.nunique() * 100
CI = get_confidence_interval(perc_shipper_to_3p, ss=len(df), pop=df_prod.asin.nunique())
perc_shipper_to_3p, CI
# perc_shipper_to_3p - CI, perc_shipper_to_3p + CI

(6.609605325725154, 1.0547655730262855)

## How often do third party merchants change?

In [32]:
third_party_seller = df[df.seller_delta == 'third party']
len(third_party_seller[third_party_seller.sold_by_new != third_party_seller.sold_by_og]) / len(third_party_seller)

0.1793440334961619

In [33]:
df.sold_by_og.value_counts(normalize=True).head(20)

Amazon.com                 0.272468
Whole Foods Market         0.009510
AmazonFresh                0.008559
Amazon.com Services LLC    0.004755
Pharmapacks                0.003329
Amazon KINDLE              0.002378
Amazon Warehouse           0.001902
Zappos                     0.001902
Teslagear Inc.             0.001427
wrestling_superstore       0.001427
Jockey                     0.001427
MYBATTERYSUPPLIER          0.001427
Eddie Bauer                0.001427
recommerce                 0.001427
imagikids LLC              0.001427
bululuo                    0.001427
SilverPoint                0.001427
VM Express                 0.001427
SweatyRocks                0.001427
Miulee Home                0.001427
Name: sold_by_og, dtype: float64

## Compare to the 2016 Northeastern study
The study looked at best-selling products from multiple sellers.

In [34]:
df_ne = df[df.has_third_party_sellers_new == True]
(df_ne['sold_by_new'] == df_ne['sold_by_og']).value_counts(normalize=True)

True     0.765095
False    0.234905
dtype: float64

In [35]:
df_ne.is_sold_by_amazon_og.value_counts(normalize=True)

False    0.600496
True     0.399504
Name: is_sold_by_amazon_og, dtype: float64

In [36]:
df_ne.is_shipped_by_amazon_og.value_counts(normalize=True)

True     0.901408
False    0.098592
Name: is_shipped_by_amazon_og, dtype: float64

In [37]:
# seller changes to Amazon
perc_seller_to_amazon = df_ne[df_ne.seller_delta.isin(['to amazon', 'to amazon'])].asin.nunique() / df_ne.asin.nunique() * 100
CI = get_confidence_interval(perc_seller_to_amazon, ss=len(df_ne), pop=df_prod.asin.nunique())
perc_seller_to_amazon, CI
# perc_seller_to_amazon - CI, perc_seller_to_amazon + CI

(2.0678246484698097, 0.7990786868465768)

In [38]:
# seller changes to third-party
perc_seller_to_3p = df_ne[df_ne.seller_delta.isin(['to amazon', 'to third party'])].asin.nunique() / df_ne.asin.nunique() * 100
CI = get_confidence_interval(perc_seller_to_3p, ss=len(df_ne), pop=df_prod.asin.nunique())
perc_seller_to_3p, CI
# perc_seller_to_3p - CI, perc_seller_to_3p + CI

(4.383788254755997, 1.1496364992860233)

In [39]:
# shipper changes to Amazon
perc_shipper_to_amazon = df_ne[df_ne.shipper_delta.isin(['to amazon', 'to amazon'])].asin.nunique() / df.asin.nunique() * 100
CI = get_confidence_interval(perc_shipper_to_amazon, ss=len(df_ne),pop=df_prod.asin.nunique())
perc_shipper_to_amazon, CI
# perc_shipper_to_amazon - CI, perc_shipper_to_amazon + CI

(2.3300047551117453, 0.8470888778157493)

In [40]:
# shipper changes to third-party
perc_shipper_to_3p = df_ne[df_ne.shipper_delta.isin(['to amazon', 'to third party'])].asin.nunique() / df_ne.asin.nunique() * 100
CI = get_confidence_interval(perc_shipper_to_3p, ss=len(df_ne), pop=df_prod.asin.nunique())
perc_shipper_to_3p, CI
# perc_shipper_to_3p - CI, perc_shipper_to_3p + CI

(7.775020678246484, 1.5036440919325724)

In [41]:
third_party_seller = df_ne[df_ne.seller_delta == 'third party']
len(third_party_seller[third_party_seller.sold_by_new != third_party_seller.sold_by_og]) / len(third_party_seller)

0.31383737517831667

In [42]:
df_ne.sold_by_og.value_counts(normalize=True).head(20)

Amazon.com            0.375517
AmazonFresh           0.007444
Pharmapacks           0.005790
Whole Foods Market    0.004963
Amazon KINDLE         0.004136
Zappos                0.003309
SilverPoint           0.002481
recommerce            0.002481
Amazon Warehouse      0.002481
MYBATTERYSUPPLIER     0.002481
SilicaGelProducts     0.001654
Toy Planet            0.001654
Heartland Food        0.001654
MAV Supply            0.001654
imagikids LLC         0.001654
KAMANINA US           0.001654
YuLang-US             0.001654
RGM GROUP             0.001654
VeeTrends             0.001654
NetRush               0.001654
Name: sold_by_og, dtype: float64